In [1]:
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader , random_split
import torch.nn.functional as F

In [2]:
import json

dataset = json.load(open('Robert_Frost_Poem_Collections.json'))

titles = []
contents = []
for item in dataset:
  titles.append(item.get('title'))
  contents.append(item.get('content'))

training_text = ''.join(contents)

with open('training_text.txt', 'w+', encoding='utf-8') as f:
    f.write(training_text)

In [3]:
import sentencepiece as spm

class Tokenizer:
    def __init__(self, vocab_size= 3000, model_file='spm_vocab.model'):
        self.vocab_size = vocab_size
        self.model_file = model_file
        self.special_tokens = {
            '<PADDING>': 0,
            '<STARTLINE>': 1,
            '<ENDLINE>': 2,
            '<UNKNOWN>': 3
        }
        self.tokenizer = None
        self.train()
        self.tokenizer = spm.SentencePieceProcessor(model_file=self.model_file)     # type: ignore
        self.max_pad_length = self._find_max_token_length('training_text.txt')
        self._build_vocab()

    def train(self):
        spm.SentencePieceTrainer.train(                                             # type: ignore
            input='training_text.txt',
            model_prefix='spm_vocab',
            vocab_size=self.vocab_size,
            model_type='bpe',
            pad_id=self.special_tokens['<PADDING>'],
            unk_id=self.special_tokens['<UNKNOWN>'],
            bos_id=self.special_tokens['<STARTLINE>'],
            eos_id=self.special_tokens['<ENDLINE>'],
            user_defined_symbols=list(self.special_tokens.keys())
        )

    def _find_max_token_length(self, file_path):
        max_length = 0

        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                tokens = self.tokenizer.encode_as_pieces(line)                     # type: ignore
                max_length = max(max_length, len(tokens))

        return max(max_length,40)

    def _build_vocab(self):
        self.vocab = {token: idx for token, idx in self.special_tokens.items()}

        for id in range(self.tokenizer.get_piece_size()):                           # type: ignore
            piece = self.tokenizer.id_to_piece(id)                                  # type: ignore
            if piece not in self.vocab:
                self.vocab[piece] = len(self.vocab)

        self.reverse_vocab = {v: k for k, v in self.vocab.items()}

    def encode(self, text, method="int", pad_output=False):
        if method == "int": encoded = [self.special_tokens['<STARTLINE>']] + self.tokenizer.encode_as_ids(text) + [self.special_tokens['<ENDLINE>']]       # type: ignore
        else: encoded = ['<STARTLINE>'] + self.tokenizer.encode_as_pieces(text) + ['<ENDLINE>']             # type: ignore


        if pad_output:
            encoded = encoded[:self.max_pad_length]
            padding_needed = self.max_pad_length - len(encoded)
            if padding_needed > 0:
                if (method == "int") : encoded += [self.special_tokens['<PADDING>']] * padding_needed
                else : encoded += ['<PADDING>'] * padding_needed

        return encoded


    def decode(self, ids, method="int"):
        if method == "int":
            ids = [id for id in ids if id not in [self.special_tokens['<STARTLINE>'], self.special_tokens['<ENDLINE>']]]
            return self.tokenizer.decode_ids(ids)                                                                           # type: ignore
        else:
            pieces = [piece for piece in ids if piece not in ['<STARTLINE>', '<ENDLINE>', '<PADDING>']]
            return self.tokenizer.decode_pieces(pieces)                                                                     # type: ignore

    def create_input_output(self, dataset):
        inputs = []

        for lines in dataset:
            encoded_sentences = self.encode(lines, pad_output=True)
            inputs.append(torch.tensor(encoded_sentences, dtype=torch.long))

        inputs = torch.stack(inputs)

        return inputs

In [4]:
tokenizer = Tokenizer(vocab_size=3000)

Text_Dataset = [lines for content in contents for lines in content.split('\r\n')]

inp = tokenizer.create_input_output(Text_Dataset)
print(f"Input Data shape is  : {inp.shape}")

train_size = int(0.8 * len(inp))
test_size = len(inp) - train_size
train_data, test_data = random_split(inp, [train_size, test_size])          # type: ignore

train_dataloader = DataLoader(train_data, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=16, shuffle=False)

Input Data shape is  : torch.Size([2507, 40])


sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input: training_text.txt
  input_format: 
  model_prefix: spm_vocab
  model_type: BPE
  vocab_size: 3000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  user_defined_symbols: <PADDING>
  user_defined_symbols: <STARTLINE>
  user_defined_symbols: <ENDLINE>
  user_defined_symbols: <UNKNOWN>
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 3
  bos_id: 1
  eos_id: 2
  pad

In [25]:
class Predictor(nn.Module):
    def __init__(self, input_size, hidden_size, layers, output_size,emb_size = 512):
        super(Predictor, self).__init__()

        self.embedding = nn.Embedding(input_size,emb_size)
        self.mlb = nn.Sequential(nn.Linear(emb_size,emb_size), nn.LayerNorm(emb_size) , nn.ELU() , nn.Linear(emb_size,emb_size))
        self.lstm = nn.LSTM(emb_size, hidden_size, num_layers=layers, batch_first=True, dropout=0.2)
        self.out = nn.Sequential(nn.Linear(hidden_size,hidden_size//2), nn.LayerNorm(hidden_size//2), nn.ELU(), nn.Dropout(0.5), nn.Linear(hidden_size//2 , output_size))

    def forward(self, input_seq, hidden, mem):
        input_emb = self.embedding(input_seq)
        input_emb = self.mlb(input_emb)
        output , (hidden_out , memory_out) = self.lstm(input_emb,(hidden,mem))

        return self.out(output) , hidden_out , memory_out

In [26]:
input_size = tokenizer.vocab_size
output_size = tokenizer.vocab_size
hidden_size = 256
layers = 2
epochs = 10
batch_size = 16

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Predictor(input_size, hidden_size, layers, output_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)                #type: ignore
loss_fn = nn.CrossEntropyLoss()
lost_logs = []

for epoch in range(epochs):
    steps = 0
    model.train()
    loss = 0

    for text in train_dataloader:
        input = text[:,0:-1].to(device)
        output = text[:,1:].to(device)

        hidden = torch.zeros(layers, input.shape[0], hidden_size).to(device)
        cell = torch.zeros(layers, input.shape[0], hidden_size).to(device)

        pred,hidden,cell = model(input, hidden, cell)
        pred = pred.transpose(1,2)
        pred = pred.float()
    
        loss = loss_fn(pred, output)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        lost_logs.append(loss)

In [28]:
import numpy as np

def evaluate(model, dataloader, loss_fn, device):
    model.eval() 
    total_loss = 0
    total_correct = 0
    total_tokens = 0
    total_examples = 0
    
    with torch.no_grad():  
        for text in dataloader:
            input_seq = text[:, 0:-1].to(device)  
            target_seq = text[:, 1:].to(device)  

            hidden = torch.zeros(layers, input_seq.shape[0], hidden_size).to(device)
            cell = torch.zeros(layers, input_seq.shape[0], hidden_size).to(device)

            predictions, hidden, cell = model(input_seq, hidden, cell)

            predictions = predictions.transpose(1, 2).float()
            loss = loss_fn(predictions, target_seq)

            total_loss += loss.item()

            _, predicted_indices = torch.max(predictions, dim=1)  
            total_correct += (predicted_indices == target_seq).sum().item() 
            total_tokens += target_seq.numel()  
            total_examples += input_seq.shape[0]  

    avg_loss = total_loss / len(dataloader)
    accuracy = total_correct / total_tokens
    perplexity = np.exp(avg_loss)

    return avg_loss, accuracy, perplexity

avg_loss, accuracy, perplexity = evaluate(model, test_dataloader, loss_fn, device)

print(f"Evaluation Results:")
print(f"Average Loss: {avg_loss:.4f}")
print(f"Accuracy: {accuracy*100:.2f}%")
print(f"Perplexity: {perplexity:.2f}")


Evaluation Results:
Average Loss: 1.7506
Accuracy: 72.65%
Perplexity: 5.76


In [46]:
def sample_from_logits(logits, temperature):
    logits = logits / temperature
    probs = F.softmax(logits, dim=-1)
    return torch.multinomial(probs, num_samples=1).item()

def generate_text(model, start_sequence, max_length, device, temperature=1.0):
    model.eval()
    generated_sequence = start_sequence
    input_seq = torch.tensor(start_sequence).unsqueeze(0).to(device)
    hidden = torch.zeros(layers, 1, hidden_size).to(device)
    mem = torch.zeros(layers, 1, hidden_size).to(device)

    for _ in range(max_length):
        with torch.no_grad(): output, hidden, mem = model(input_seq, hidden, mem)
        
        logits = output[:, -1, :]
    
        predicted_token = sample_from_logits(logits, temperature)
        generated_sequence.append(predicted_token)
        input_seq = torch.tensor(generated_sequence[-len(start_sequence):]).unsqueeze(0).to(device)

    return generated_sequence

max_length = 100  
temperature = 0.6
poem = []

for _ in range(5):
    start_sequence = [tokenizer.vocab.get('<STARTLINE>')]  
    generated_tokens = generate_text(model, start_sequence, max_length, device, temperature)
    poem.append(tokenizer.decode(generated_tokens))

for lines in poem : print(lines)

"I'm not afraid of what you're going to say.
I'm not afraid of it, or you're you.
On the windows that one man's all
And there's I don't know what he's not safe
I'm just there. I've not.
